# IMPORT BIBLIOTEK

In [1]:
import mysql.connector
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta


# POŁĄCZENIE Z BAZĄ DANYCH

In [2]:

con = mysql.connector.connect(
    host = "giniewicz.it",
    user = "team07",
    password = "te@mlot",
    database = "team07"
)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

Połączenie udane


# TWORZENIE SCHEMATU BAZY DANYCH

In [ ]:
cursor = con.cursor()

schema_sql = """
-- Wyłączenie sprawdzania kluczy obcych
SET FOREIGN_KEY_CHECKS = 0;

-- Usunięcie wszystkich tabel, jeśli istnieją
DROP TABLE IF EXISTS TripParticipants;
DROP TABLE IF EXISTS Transactions;
DROP TABLE IF EXISTS TripCrew;
DROP TABLE IF EXISTS RecoveryTasks;
DROP TABLE IF EXISTS RepairTasks;
DROP TABLE IF EXISTS Trips;
DROP TABLE IF EXISTS TripOffers;
DROP TABLE IF EXISTS Customers;
DROP TABLE IF EXISTS Employees;
DROP TABLE IF EXISTS Rockets;
DROP TABLE IF EXISTS Spaceships;
DROP TABLE IF EXISTS Destinations;

-- Włączenie sprawdzania kluczy obcych
SET FOREIGN_KEY_CHECKS = 1;

-- Tworzenie tabeli Customers
CREATE TABLE Customers (
  pesel VARCHAR(11) PRIMARY KEY,
  first_name VARCHAR(100),
  last_name VARCHAR(100),
  date_of_birth DATE,
  is_insured BOOLEAN,
  email VARCHAR(150),
  phone_number VARCHAR(30),
  emergency_contact_name VARCHAR(100),
  emergency_contact_phone VARCHAR(30),
  emergency_contact_relation VARCHAR(50)
);

-- Tworzenie tabeli Rockets
CREATE TABLE Rockets (
  rocket_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  manufacturing_date DATE,
  status VARCHAR(50),
  initial_cost DECIMAL(12,2)
);

-- Tworzenie tabeli Spaceships
CREATE TABLE Spaceships (
  spaceship_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  capacity INTEGER,
  travel_speed FLOAT
  manufacturing_date DATE,
  status VARCHAR(50),
  initial_cost DECIMAL(12,2)
);

-- Tworzenie tabeli Employees
CREATE TABLE Employees (
  pesel VARCHAR(11) PRIMARY KEY,
  first_name VARCHAR(100),
  last_name VARCHAR(100),
  sex CHAR(1),
  date_of_birth DATE,
  role VARCHAR(100),
  specialization VARCHAR(100),
  salary DECIMAL(10,2)
);

-- Tworzenie tabeli Destinations
CREATE TABLE Destinations (
  destination_id INTEGER PRIMARY KEY,
  name VARCHAR(100),
  description TEXT,
  distance_range VARCHAR(100)
);

-- Tworzenie tabeli TripOffers
CREATE TABLE TripOffers (
  trip_type_id INTEGER PRIMARY KEY,
  destination_id INTEGER,
  name VARCHAR(100),
  description TEXT,
  launch_date DATE,
  return_date DATE,
  duration_of_stay INTEGER,
  FOREIGN KEY (destination_id) REFERENCES Destinations(destination_id)
);

-- Tworzenie tabeli Trips
CREATE TABLE Trips (
  trip_id INTEGER PRIMARY KEY,
  trip_type_id INTEGER,
  destination_id INTEGER,
  launch_date DATE,
  return_date DATE,
  spaceship_id INTEGER,
  rocket_id INTEGER,
  status VARCHAR(50),
  organization_cost DECIMAL(12,2),
  FOREIGN KEY (trip_type_id) REFERENCES TripOffers(trip_type_id),
  FOREIGN KEY (destination_id) REFERENCES Destinations(destination_id),
  FOREIGN KEY (spaceship_id) REFERENCES Spaceships(spaceship_id),
  FOREIGN KEY (rocket_id) REFERENCES Rockets(rocket_id)
);

-- Tworzenie tabeli TripParticipants
CREATE TABLE TripParticipants (
  participant_id INTEGER PRIMARY KEY,
  trip_id INTEGER,
  customer_id VARCHAR(11),
  ticket_price DECIMAL(10,2),
  status VARCHAR(20),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id),
  FOREIGN KEY (customer_id) REFERENCES Customers(pesel)
);

-- Tworzenie tabeli Transactions
CREATE TABLE Transactions (
  transaction_id INTEGER PRIMARY KEY,
  customer_id VARCHAR(11),
  amount DECIMAL(10,2),
  transaction_date TIMESTAMP,
  method VARCHAR(50),
  FOREIGN KEY (customer_id) REFERENCES Customers(pesel)
);

-- Tworzenie tabeli TripCrew
CREATE TABLE TripCrew (
  trip_crew_id INTEGER PRIMARY KEY,
  trip_id INTEGER,
  employee_id VARCHAR(11),
  role VARCHAR(50),
  FOREIGN KEY (trip_id) REFERENCES Trips(trip_id),
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel)
);

-- Tworzenie tabeli RecoveryTasks
CREATE TABLE RecoveryTasks (
  recovery_task_id INTEGER PRIMARY KEY,
  employee_id VARCHAR(11),
  rocket_id INTEGER,
  start_date DATE,
  end_date DATE,
  status VARCHAR(50),
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel),
  FOREIGN KEY (rocket_id) REFERENCES Rockets(rocket_id)
);

-- Tworzenie tabeli RepairTasks
CREATE TABLE RepairTasks (
  repair_task_id INTEGER PRIMARY KEY,
  employee_id VARCHAR(11),
  spaceship_id INTEGER,
  start_date DATE,
  end_date DATE,
  status VARCHAR(50),
  FOREIGN KEY (employee_id) REFERENCES Employees(pesel),
  FOREIGN KEY (spaceship_id) REFERENCES Spaceships(spaceship_id)
);
"""
for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement + ';')

con.commit()
print("Schemat bazy danych został utworzony.")

Schemat bazy danych został utworzony.


### DODANIE CELÓW PODRÓŻY

In [5]:
cursor = con.cursor()

schema_sql = """
-- Wstawianie danych do tabeli Destinations
INSERT INTO Destinations (destination_id, name, description, distance_range)
VALUES
  (1, 'Mercury', 'Najbliższa Słońcu planeta, znana z ekstremalnych temperatur i surowego krajobrazu.', '0.61-1.47 AU'),
  (2, 'Venus', 'Gorąca planeta z gęstą atmosferą, często nazywana bliźniaczką Ziemi.', '0.27-1.73 AU'),
  (3, 'Mars', 'Czerwona planeta z potencjalnymi śladami wody i celem przyszłych misji kolonizacyjnych.', '0.52-2.67 AU'),
  (4, 'Moon', 'Naturalny satelita Ziemi, oferujący spektakularne widoki i historyczne miejsca lądowań.', '0.00257 AU'),
  (5, 'Space Station', 'Orbitująca stacja kosmiczna na niskiej orbicie ziemskiej, idealna dla budżetowych wycieczek.', '0.00000267 AU');
"""
for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement + ';')

con.commit()
print("Dane do tabeli Destinations zostały wstawione.")

Dane do tabeli Destinations zostały wstawione.


### Dodanie rakiet i statków kosmicznych

In [18]:
cursor = con.cursor()

schema_sql = """
-- Usuwanie istniejących danych z tabel
DELETE FROM Rockets;
DELETE FROM Spaceships;

-- Wstawianie danych do tabeli Rockets
INSERT INTO Rockets (rocket_id, name, manufacturing_date, status, initial_cost)
VALUES
  (1, 'OrbitalLift-1', '2165-03-15', 'Active', 5000000.00),
  (2, 'OrbitalLift-2', '2165-04-20', 'Active', 5000000.00),
  (3, 'OrbitalLift-3', '2165-05-25', 'Active', 5000000.00),
  (4, 'OrbitalLift-4', '2165-06-30', 'Active', 5000000.00),
  (5, 'LunarExplorer-1', '2166-01-10', 'Active', 15000000.00),
  (6, 'LunarExplorer-2', '2166-02-15', 'Active', 15000000.00),
  (7, 'LunarExplorer-3', '2166-03-20', 'Active', 15000000.00),
  (8, 'LunarExplorer-4', '2166-04-25', 'Active', 15000000.00),
  (9, 'InterplanetaryVoyager-1', '2167-01-15', 'Active', 50000000.00),
  (10, 'InterplanetaryVoyager-2', '2167-02-20', 'Active', 50000000.00),
  (11, 'InterplanetaryVoyager-3', '2167-03-25', 'Active', 50000000.00),
  (12, 'InterplanetaryVoyager-4', '2167-04-30', 'Active', 50000000.00);

-- Wstawianie danych do tabeli Spaceships
INSERT INTO Spaceships (spaceship_id, name, capacity, travel_speed, manufacturing_date, status, initial_cost)
VALUES
  (1, 'OrbitalShuttle-1', 5, 7.8, '2165-03-20', 'Active', 3000000.00),
  (2, 'OrbitalShuttle-2', 5, 7.8, '2165-04-25', 'Active', 3000000.00),
  (3, 'OrbitalShuttle-3', 5, 7.8, '2165-05-30', 'Active', 3000000.00),
  (4, 'OrbitalShuttle-4', 5, 7.8, '2165-06-15', 'Active', 3000000.00),
  (5, 'LunarCruiser-1', 10, 10.5, '2166-01-20', 'Active', 12000000.00),
  (6, 'LunarCruiser-2', 10, 10.5, '2166-02-25', 'Active', 12000000.00),
  (7, 'LunarCruiser-3', 10, 10.5, '2166-03-30', 'Active', 12000000.00),
  (8, 'LunarCruiser-4', 10, 10.5, '2166-04-15', 'Active', 12000000.00),
  (9, 'InterplanetaryArk-1', 20, 15.0, '2167-01-25', 'Active', 40000000.00),
  (10, 'InterplanetaryArk-2', 20, 15.0, '2167-02-28', 'Active', 40000000.00),
  (11, 'InterplanetaryArk-3', 20, 15.0, '2167-03-15', 'Active', 40000000.00),
  (12, 'InterplanetaryArk-4', 20, 15.0, '2167-04-20', 'Active', 40000000.00);
"""

for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement)
con.commit()
print("Dane do tabel Rockets i Spaceships zostały zaktualizowane.")

Dane do tabel Rockets i Spaceships zostały zaktualizowane.


### Dodanie pracowników

In [11]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta
import random

# Funkcje do generowania PESEL i daty urodzenia
def losowa_data_urodzenia():
    start = datetime(2117, 1, 1)  # Maksymalny wiek 50 lat w 2167
    end = datetime(2146, 12, 31)  # Minimalny wiek 21 lat w 2167
    delta = end - start
    random_days = random.randint(0, delta.days)
    return (start + timedelta(days=random_days)).date()

def generuj_pesel(data_urodzenia, gender):
    rok = data_urodzenia.year
    miesiac = data_urodzenia.month
    dzien = data_urodzenia.day
    
    # Kodowanie miesiąca dla lat 2100–2199
    if 2100 <= rok <= 2199:
        miesiac += 40
    
    # Cyfra płci: nieparzysta dla mężczyzn, parzysta dla kobiet
    if gender == 'M':
        cyfra_plci = random.choice([1, 3, 5, 7, 9])
    else:
        cyfra_plci = random.choice([0, 2, 4, 6, 8])
    
    losowa_czesc = f"{random.randint(0, 999):03d}{cyfra_plci}"
    pesel_bez_kontrolnej = f"{rok % 100:02d}{miesiac:02d}{dzien:02d}{losowa_czesc}"
    
    wagi = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3]
    suma = sum(int(pesel_bez_kontrolnej[i]) * wagi[i] for i in range(10))
    cyfra_kontrolna = (10 - (suma % 10)) % 10
    
    return f"{pesel_bez_kontrolnej}{cyfra_kontrolna}"

# Wczytanie danych z plików CSV
imiona_men_df = pd.read_csv('../data/imiona_men.csv')
nazwiska_men_df = pd.read_csv('../data/nazwiska_men.csv')
imiona_woman_df = pd.read_csv('../data/imiona_woman.csv')
nazwiska_woman_df = pd.read_csv('../data/nazwiska_woman.csv')

# Zakładam, że pliki CSV mają kolumny 'imie' i 'nazwisko'
men_names = imiona_men_df['imie'].tolist()
men_surnames = nazwiska_men_df['nazwisko'].tolist()
women_names = imiona_woman_df['imie'].tolist()
women_surnames = nazwiska_woman_df['nazwisko'].tolist()

# Struktura korporacji
employees_structure = [
    {'role': 'CEO', 'specialization': 'executive', 'count': 1, 'salary': 5000000.00},
    {'role': 'department_head_service', 'specialization': 'service_management', 'count': 1, 'salary': 500000.00},
    {'role': 'department_head_pilot', 'specialization': 'pilot_management', 'count': 1, 'salary': 500000.00},
    {'role': 'department_head_mechanic', 'specialization': 'mechanic_management', 'count': 1, 'salary': 500000.00},
    {'role': 'department_head_admin', 'specialization': 'admin_management', 'count': 1, 'salary': 500000.00},
    {'role': 'service_staff', 'specialization': 'service', 'count': 100, 'salary': 80000.00},
    {'role': 'pilot', 'specialization': 'pilot', 'count': 20, 'salary': 150000.00},
    {'role': 'mechanic', 'specialization': 'mechanic', 'count': 30, 'salary': 100000.00},
    {'role': 'admin', 'specialization': 'administration', 'count': 40, 'salary': 70000.00},
    {'role': 'space_airport_staff', 'specialization': 'airport_operations', 'count': 1000, 'salary': 50000.00}
]

# Generowanie danych pracowników
employees = []
used_pesels = set()  # Zbiór do śledzenia unikalnych numerów PESEL

for emp in employees_structure:
    role = emp['role']
    specialization = emp['specialization']
    count = emp['count']
    salary = emp['salary']
    
    for _ in range(count):
        # Losowanie płci (50% mężczyzn, 50% kobiet)
        gender = np.random.choice(['M', 'F'])
        if gender == 'M':
            first_name = np.random.choice(men_names)
            last_name = np.random.choice(men_surnames)
        else:
            first_name = np.random.choice(women_names)
            last_name = np.random.choice(women_surnames)
        
        # Generowanie daty urodzenia i unikalnego PESEL
        while True:
            date_of_birth = losowa_data_urodzenia()
            pesel = generuj_pesel(date_of_birth, gender)
            if pesel not in used_pesels:
                used_pesels.add(pesel)
                break
        
        employees.append({
            'pesel': pesel,
            'first_name': first_name,
            'last_name': last_name,
            'sex': gender,
            'date_of_birth': date_of_birth,
            'role': role,
            'specialization': specialization,
            'salary': salary
        })

# Połączenie z bazą danych i wstawianie danych
try:
    con = mysql.connector.connect(
        host = "giniewicz.it",
        user = "team07",
        password = "te@mlot",
        database = "team07"  
    )
    if con.is_connected():
        cursor = con.cursor()
        
        # Usunięcie istniejących rekordów z tabeli Employees
        cursor.execute("DELETE FROM Employees;")
        
        # Definicja rozmiaru partii
        batch_size = 50
        
        # Wstawianie rekordów w partiach za pomocą executemany
        for i in range(0, len(employees), batch_size):
            batch = employees[i:i + batch_size]
            # Przygotowanie danych jako listy krotek
            data = [
                (
                    emp['pesel'],
                    emp['first_name'],
                    emp['last_name'],
                    emp['sex'],
                    emp['date_of_birth'],
                    emp['role'],
                    emp['specialization'],
                    emp['salary']
                ) for emp in batch
            ]
            try:
                # Wykonanie zapytania parametryzowanego z poprawnymi placeholderami
                cursor.executemany(
                    "INSERT INTO Employees (pesel, first_name, last_name, sex, date_of_birth, role, specialization, salary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                    data
                )
                print(f"Inserted batch {i // batch_size + 1} with {len(batch)} records.")
            except Error as e:
                print(f"Błąd podczas wstawiania partii {i // batch_size + 1}: {e}")
        
        # Zatwierdzenie zmian w bazie danych
        con.commit()
        print("Dane do tabeli Employees zostały wstawione.")
        
except Error as e:
    print(f"Błąd połączenia z bazą danych: {e}")
finally:
    if con.is_connected():
        cursor.close()
        con.close()
        print("Połączenie z bazą danych zostało zamknięte.")

KeyboardInterrupt: 

In [14]:

con = mysql.connector.connect(
    host = "giniewicz.it",
    user = "team07",
    password = "te@mlot",
    database = "team07"
)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

Połączenie udane


### DODANIE TRIP OFFERS

In [15]:

cursor = con.cursor()

schema_sql = """
-- Wstawianie danych do tabeli TripOffers
INSERT INTO TripOffers (trip_type_id, destination_id, name, description, launch_date, return_date, duration_of_stay)
VALUES
  (1, 1, '1-dniowy spacer po Księżycu', 'Krótka wycieczka z spacerem po powierzchni Księżyca', NULL, NULL, 1),
  (2, 1, '3-dniowa eksploracja Księżyca', 'Wycieczka z eksploracją kraterów i baz księżycowych', NULL, NULL, 3),
  (3, 2, 'Dwutygodniowy obóz na Marsie', 'Obóz survivalowy z eksploracją Czerwonej Planety', NULL, NULL, 14),
  (4, 2, '5-dniowa misja na Marsie', 'Krótka misja z wizytą w bazie mariańskiej', NULL, NULL, 5),
  (5, 3, 'Tygodniowa przygoda na Wenus', 'Wycieczka z ochroną przed wysokimi temperaturami', NULL, NULL, 7),
  (6, 3, '3-dniowy przelot nad Wenus', 'Podniebna wycieczka z widokiem na atmosferę Wenus', NULL, NULL, 3),
  (7, 4, '2-dniowy pobyt na Stacji Kosmicznej', 'Odwiedziny na orbicie z treningiem astronautycznym', NULL, NULL, 2),
  (8, 4, '4-dniowy wypoczynek na Stacji Kosmicznej', 'Relaks i obserwacje Ziemi z orbity', NULL, NULL, 4),
  (9, 5, '10-dniowa ekspedycja na Merkurego', 'Wycieczka zbadaj najgorętszą planetę Układu Słonecznego', NULL, NULL, 10),
  (10, 5, '6-dniowa misja na Merkurego', 'Krótka misja z obserwacjami powierzchni Merkurego', NULL, NULL, 6),
  (11, 1, 'Weekend na Księżycu', '2-dniowa wycieczka z noclegiem na Księżycu', NULL, NULL, 2),
  (12, 2, 'Miesiąc na Marsie', 'Długoterminowy pobyt z naukowymi eksperymentami', NULL, NULL, 30),
  (13, 3, '8-dniowa wyprawa na Wenus', 'Wycieczka z badaniem geologii Wenus', NULL, NULL, 8),
  (14, 4, '3-dniowy kurs astronautyczny', 'Szkolenie na Stacji Kosmicznej', NULL, NULL, 3),
  (15, 5, '12-dniowa eksploracja Merkurego', 'Rozszerzona misja z analizą kraterów', NULL, NULL, 12);
"""


for statement in schema_sql.strip().split(';'):
    if statement.strip():
        cursor.execute(statement)
con.commit()
print("dane do tabeli TripOffers zostały wstawione.")


dane do tabeli TripOffers zostały wstawione.


## WYGENEROWANIE TRIP OFFERS

### WYGENEROWANIE TRIPÓW

### WYGENEROWAĆ KLIENTÓW DO WYBRANYCH TRIPÓW